# 🎬 Movie Recommendation Notebook

Basic EDA and recommendation system.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

movies = pd.read_csv('../data/movies.csv')
ratings = pd.read_csv('../data/ratings.csv')
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Collaborative Filtering
user_movie_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating').fillna(0)
similarity_matrix = cosine_similarity(user_movie_matrix.T)
movie_similarity_df = pd.DataFrame(similarity_matrix, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

# Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])
content_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

def get_movie_recommendations(movie_title, similarity_df, top_n=3):
    scores = similarity_df[movie_title].sort_values(ascending=False)[1:top_n+1]
    return scores

def get_movie_recommendations_with_sentiment(movie_title, similarity_df, movie_sentiments, movies_df, top_n=3):
    scores = similarity_df[movie_title].sort_values(ascending=False)[1:]
    recommendations = []
    for movie in scores.index:
        movie_id = movies_df[movies_df['title'] == movie]['movieId'].values[0]
        sentiment = movie_sentiments[movie_sentiments['movieId'] == movie_id]['sentiment'].values
        if len(sentiment) > 0 and sentiment[0] > 0:
            recommendations.append(movie)
        if len(recommendations) >= top_n:
            break
    return recommendations